In [6]:
import lancedb
db_path = "E:\\PythonProject\\LLM-Explore\\src\\agent\\contact-query\\tmp\\contact_vectors.lancedb"
# --- 1. 连接/创建 LanceDB 数据库 ---
db = lancedb.connect(db_path)
print(f"LanceDB connected at: {db.uri}")

LanceDB connected at: E:\PythonProject\LLM-Explore\src\agent\contact-query\tmp\contact_vectors.lancedb


In [7]:
# --- 2. 打开表 ---
table_name = "contact_table" # 你之前创建的表名
try:
    table = db.open_table(table_name)
    print(f"Successfully opened table: '{table.name}'")
    print(f"Table schema: {table.schema}")
    print(f"Number of rows in table: {len(table)}")
except Exception as e:
    print(f"Error opening table '{table_name}': {e}")
    print(f"Available tables: {db.table_names()}")
    exit()

Successfully opened table: 'contact_table'
Table schema: vector: fixed_size_list<item: float>[1024]
  child 0, item: float
id: string
payload: string
Number of rows in table: 3


In [8]:
import json

def read_and_decode_table(db, table_name):
    table = db.open_table(table_name)
    df = table.to_pandas()
    
    if 'payload' not in df.columns:
        raise ValueError("DataFrame中缺少'payload'列")
    
    # 解析payload字符串为字典
    df['parsed_payload'] = df['payload'].apply(
        lambda x: json.loads(x) if isinstance(x, str) else x
    )
    
    # 直接提取已解码的字段值
    df['name_cn'] = df['parsed_payload'].apply(
        lambda x: x.get('name') if isinstance(x, dict) else None
    )
    
    df['content_cn'] = df['parsed_payload'].apply(
        lambda x: x.get('content') if isinstance(x, dict) else None
    )
    
    # 可选：删除临时解析列
    df = df.drop(columns=['parsed_payload'])
    
    return df

In [15]:
data = read_and_decode_table(db, table_name)

In [16]:
data

,vector,id,payload,name_cn,content_cn
0,"[-0.055652622, 0.017284866, -0.06402377, 0.001...",d5357d4bf0b710e3171671a07d9d3ea4,"{""name"": null, ""meta_data"": {}, ""content"": ""**...",None,**买方合同编号：BGYRHS-2023-05**\n\n**卖方合同编号：**\n\n订 ...
1,"[-0.06340503, 0.016529335, -0.036046136, 0.011...",809df58a2a389d9b13c70955cdae763c,"{""name"": null, ""meta_data"": {}, ""content"": ""\u...",None,项目名称：包头钢铁（集团）有限责任公司新体系2×500m2烧结余热利用项目\n\n汽轮机转子...
2,"[-0.031060582, 0.01269974, -0.03641976, 0.0231...",36e98b1ae348a3b881d88b6249b6e6f4,"{""name"": null, ""meta_data"": {}, ""content"": ""\u...",None,买方合同号：BTYR-E-2012-001\n\n卖方合同号：南汽电合字2012第036号\...
3,"[-0.034840934, 0.012854773, -0.04900703, 0.053...",39de3bc609ad413351b5d1204058499d,"{""name"": null, ""meta_data"": {}, ""content"": ""**...",None,**买方合同编号：BTYR-E-2012-002**\n\n**卖方合同编号：**\n\n*...
4,"[-0.045348153, 0.016085029, -0.044456694, -0.0...",db9259eac52ea333547537712ed6684d,"{""name"": null, ""meta_data"": {}, ""content"": ""**...",None,**买方合同编号：BGDSYR-2020-02**\n\n**卖方合同编号：**\n\n订 ...
